In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch
!pip install tqdm
!pip install openai-clip
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 35.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 3.9 MB/s eta 0:00:00
  Created wheel for openai-clip: filename=openai_clip-1.0.1-py3-none-any.whl size=1368605 sha256=e649482528e0421553cd8aaef86bf64b05bd298db6afade1d76173f0edea60ef
  Stored in directory: /root/.cache/pip/wheels/08/77/8e/8d2f862df6bf7fb4e2007062d2cbaeae49862ec7b56d041229
Successfully built openai-clip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pya

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os
import numpy as np


In [4]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [5]:
import pathlib
class CustomGarbageDataset(Dataset):
    def __init__(self, data_dir, processor, transform=None):
        self.data_dir = pathlib.Path(data_dir)
        self.processor = processor
        self.transform = transform

        self.image_paths = []
        self.labels = []
        self.label_to_idx = {}

        # Iterate over folders for each class (assumes folder names are class labels)
        for idx, label in enumerate(sorted(os.listdir(self.data_dir))):
            label_dir = self.data_dir / label
            if label_dir.is_dir():
                for img_file in os.listdir(label_dir):
                    if img_file.endswith('.jpg') or img_file.endswith('.png'):
                        self.image_paths.append(label_dir / img_file)
                        self.labels.append(idx)
                self.label_to_idx[label] = idx

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
dataset = CustomGarbageDataset(data_dir="/content/drive/MyDrive/LightWaste", processor=processor, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [8]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
classifier = torch.nn.Linear(512, len(dataset.label_to_idx))  # Assuming CLIP embedding is 512-dim

optimizer = torch.optim.Adam(model.parameters(), lr=5e-6)
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e

In [9]:
import torch.nn.functional as F

num_epochs = 5
model.to(device)
classifier.to(device)

for epoch in range(num_epochs):
    model.train()  # Ensure the model is in training mode
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for batch in dataloader:
        # Get input tensors and labels
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        # Forward pass (allow the base model to learn by removing torch.no_grad())
        image_encodings = model.get_image_features(images)  # No normalization here
        image_encodings = image_encodings.float()

        outputs = classifier(image_encodings)  # Classifier makes predictions
        loss = criterion(outputs, labels)  # Compute loss

        # Backward pass
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total_predictions += labels.size(0)
        correct_predictions += predicted.eq(labels).sum().item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(dataloader)}, Accuracy: {100. * correct_predictions / total_predictions}%")


Epoch [1/5], Loss: 0.9013342631359895, Accuracy: 73.98586757393352%
Epoch [2/5], Loss: 0.24073189919193586, Accuracy: 94.29468725464538%
Epoch [3/5], Loss: 0.08623403397699198, Accuracy: 98.48207275582308%
Epoch [4/5], Loss: 0.055622405201817555, Accuracy: 98.82229782779378%
Epoch [5/5], Loss: 0.06090781669287632, Accuracy: 98.5344150745878%


In [12]:
import torch
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(model, classifier, dataloader, criterion, device):
    model.eval()  # Set model to evaluation mode
    classifier.eval()

    running_loss = 0.0
    all_predictions = []
    all_labels = []

    with torch.no_grad():  # Disable gradient calculation
        for batch in dataloader:
            # Get input tensors and labels
            images, labels = batch
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            image_encodings = model.get_image_features(images)
            image_encodings = image_encodings.float()
            outputs = classifier(image_encodings)

            # Compute loss
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            # Store predictions and true labels
            _, predicted = outputs.max(1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate accuracy, precision, recall, and F1-score
    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, average='macro')  # 'macro' for multi-class
    recall = recall_score(all_labels, all_predictions, average='macro')
    f1 = f1_score(all_labels, all_predictions, average='macro')

    print(f"Evaluation Loss: {running_loss / len(dataloader)}")
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F1 Score: {f1 * 100:.2f}%")

    return accuracy, precision, recall, f1




In [14]:
evaluate_model(model, classifier, dataloader, criterion, device)

Evaluation Loss: 0.01735458188535025
Accuracy: 99.92%
Precision: 99.93%
Recall: 99.91%
F1 Score: 99.92%


(0.9992148652185292, 0.9992502448216884, 0.9990857674549873, 0.999166816916379)

In [15]:
torch.save({
    'model_state_dict': model.state_dict(),
    'classifier_state_dict': classifier.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': running_loss / len(dataloader),
}, 'clip_finetuned_model.pth')

print("Model saved to 'clip_finetuned_model.pth'")

Model saved to 'clip_finetuned_model.pth'
